The Task
Your task is two-fold:

I. Clean the files and combine them into one final DataFrame.

This dataframe should have the following columns:
- Hero (Just the name of the Hero)
- Publisher
- Gender
- Eye color
- Race
- Hair color
- Height (numeric)
- Skin color
- Alignment
- Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.
    
Hint: There is a space in "100 kg" or "52.5 cm"



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Importing the OS and JSON Modules
import os,json

## Filter ALL warnings
import warnings
warnings.filterwarnings('ignore')

# Super Hero CSV

In [2]:
# CSV File
sup_hero = pd.read_csv('Data/superhero_info.csv')
sup_hero.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
# Duplicate values
print(f'There are {sup_hero.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


Separate the Hero and Publisher

In [5]:
sup_hero[['Hero', 'Publisher']] = sup_hero['Hero|Publisher'].str.split('|', expand = True)

sup_hero.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [6]:
sup_hero = sup_hero.drop(columns=['Hero|Publisher'])
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


Separate the Height and Weight

In [7]:
## examining a single value from the coordinates col
measurements = sup_hero.loc[0,"Measurements"]
print(type(measurements))
measurements


<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [8]:
measurements = measurements.replace("'",'"')
measurements

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [9]:
## now we can use json.loads
fixed_measurements = json.loads(measurements)
print(type(fixed_measurements))
fixed_measurements


<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [10]:
## use .str.replace to replace all single quotes
sup_hero['Measurements'] = sup_hero['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
sup_hero['Measurements'] = sup_hero['Measurements'].apply(json.loads)
sup_hero['Measurements'].head()


0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [11]:
## check a single value after transformation
test_measurements = sup_hero.loc[0, 'Measurements']
print(type(test_measurements))
test_measurements


<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [12]:
ht_wt = sup_hero['Measurements'].apply(pd.Series)
ht_wt

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [13]:
# concat ht_wt with original dataframe
sup_hero = pd.concat((sup_hero, ht_wt), axis = 1)
sup_hero.head(2)


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


Remove the old Column

In [14]:
sup_hero = sup_hero.drop(columns=['Measurements'])
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [15]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      463 non-null    object
 1   Race        463 non-null    object
 2   Alignment   463 non-null    object
 3   Hair color  463 non-null    object
 4   Eye color   463 non-null    object
 5   Skin color  463 non-null    object
 6   Hero        463 non-null    object
 7   Publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


Clean Columns convert data type

In [16]:
sup_hero['Weight'] = sup_hero['Weight'].str.replace(' kg', '')
sup_hero['Weight'].head(2)

0    441.0
1     65.0
Name: Weight, dtype: object

In [17]:
sup_hero['Weight'] = sup_hero['Weight'].astype(float)

In [18]:
sup_hero['Height'] = sup_hero['Height'].str.replace(' cm', '')
sup_hero['Height'].head(2)

0    203.0
1    191.0
Name: Height, dtype: object

In [19]:
sup_hero['Height'] = sup_hero['Height'].astype(float)

In [20]:
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [21]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


# Super Powers CSV

In [22]:
# CSV File
sup_powers = pd.read_csv('Data/superhero_powers.csv')
sup_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [23]:
sup_powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [24]:
# Duplicate values
print(f'There are {sup_powers.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


In [25]:
sup_powers['powers_split'] = sup_powers['Powers'].str.split(',')


In [26]:
exploded = sup_powers.explode('powers_split')


In [27]:
cols_to_make = exploded['powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [28]:
for col in cols_to_make:
    sup_powers[col] = sup_powers['Powers'].str.contains(col)
sup_powers.head()

,hero_names,Powers,powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
# Drop the unnecessary columns
sup_powers = sup_powers.drop(columns=['Powers', 'powers_split'])
sup_powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Combine Dataframe

In [30]:
## merging with different column names
hero_df = pd.merge(sup_hero, sup_powers, left_on='Hero', right_on='hero_names')

hero_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super powers who have Super Speed to those who do not.


In [40]:
# With super speed filter
speed = hero_df['Super Speed'] == True
# dataframe with filter
speed_df = hero_df[speed]


In [46]:
hero_speed = speed_df.groupby('Hero').sum()
hero_speed

,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
Hero,,,,,,,,,,,,,,,,,,,,,
Abomination,203.0,441.0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam Strange,185.0,88.0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Air-Walker,188.0,108.0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ajax,193.0,90.0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alan Scott,180.0,90.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wonder Girl,165.0,51.0,1,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Wonder Man,188.0,171.0,0,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Wonder Woman,183.0,74.0,0,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
hero_speed['Weight'].mean()

134.14659685863873

Hero's with out Super Speed

In [33]:
# no speed powers filter
no_speed_df = hero_df[~speed]

In [51]:
# weight for no speed heros
speedless = no_speed_df.groupby('Hero').sum()
speedless

,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
Hero,,,,,,,,,,,,,,,,,,,,,
A-Bomb,203.0,441.0,0,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Abe Sapien,191.0,65.0,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Abin Sur,185.0,90.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Absorbing Man,193.0,122.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Agent Bob,178.0,81.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X-23,155.0,50.0,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
X-Man,175.0,61.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yellowjacket,183.0,83.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# avg weight for those with out speed
speedless['Weight'].mean()

102.1590909090909

2. What is the average height of heroes for each publisher?

In [44]:
publisher = hero_df.groupby('Publisher').sum()
publisher

,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
Publisher,,,,,,,,,,,,,,,,,,,,,
DC Comics,25105.5,14378.0,58,76,67,61,35,57,25,2,...,1,0,0,0,0,0,0,0,0,0
Dark Horse Comics,1946.0,1120.0,7,9,7,6,5,8,4,2,...,0,0,0,0,0,0,0,0,1,0
George Lucas,798.0,387.0,4,0,3,3,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Image Comics,211.0,405.0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Marvel Comics,56889.2,35515.0,118,181,143,123,92,140,55,14,...,3,0,0,1,2,2,1,0,0,0
Shueisha,686.0,258.0,4,4,4,4,3,0,3,1,...,1,0,0,0,0,0,0,0,0,0
Star Trek,363.0,158.0,0,1,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Team Epic TV,723.0,288.0,2,2,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
Unknown,178.0,83.0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
